In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from aux_func_V3 import *
from aux_gif2D import *
import API_Numpy
import API_TensorFlow

gpus= tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
if len(gpus)>0:
    tf.config.experimental.set_memory_growth(gpus[0], True) 

import dill
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:

nomes = ["WENO-JS", "WENO-Z" , "WENO-Z+", "WENO-D" , "WENO-ZD", "WENO-ZD+"]

# API = API_Numpy
API = API_TensorFlow
equation = euler_equation_2D
γ        = 5.0/3.0

WENOs = {}
WENOs[nomes[0]] = simulation_2D(API, equation, WENO_JS_scheme , γ=γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default)
WENOs[nomes[1]] = simulation_2D(API, equation, WENO_Z_scheme  , γ=γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default)
WENOs[nomes[2]] = simulation_2D(API, equation, WENO_Zp_scheme , γ=γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default)
WENOs[nomes[3]] = simulation_2D(API, equation, WENO_D_scheme  , γ=γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default)
WENOs[nomes[4]] = simulation_2D(API, equation, WENO_ZD_scheme , γ=γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default)
WENOs[nomes[5]] = simulation_2D(API, equation, WENO_ZDp_scheme, γ=γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default)

nome = "WENO-D"

RT_range=(960,480,320)
DM_range=(400,300,200)

In [3]:
for i in range(3):
    γ = 5.0/3.0
    N = RT_range[i]
    x, y, Δx, Δy, U0 = CondiçãoInicialRayleighTaylor_N(N, γ)

    cfl     = 0.3
    t_final = 1.95

    GhostPointsX = RayleighTaylorGhostPointsX()
    GhostPointsY = RayleighTaylorGhostPointsY(γ, dtype=dtype)

    create_data(
        'RT',
        N,
        nome,
        U0,
        WENOs[nome],
        0.01,
        t_final,
        cfl,
        Δx,
        Δy,
        GhostPointsX,
        GhostPointsY,
        RayleighTaylorGravity,
        continue_flag = False)

    create_movie(
        'RT',
        N,
        nome,
        x,
        y,
        figsize=(10,16),
        vmin=0.9,
        vmax=2.3,
        levels=15,
        xlim=None,
        ylim=(0.2,0.8))

    γ = 7.0/5.0
    N = DM_range[i]
    x0=1/6
    x, y, Δx, Δy, U0 = CondiçãoInicialDoubleMach_N(N, γ,x0)
    cfl = 0.45
    t_final = 0.2

    label=f'DoubleMach'

    GhostPointsX=DoubleMachGhostPointsX(L=3,R=3,γ=γ)
    GhostPointsY=DoubleMachGhostPointsY(L=3,R=3,γ=γ,x0=x0,x=x)

    WENOs[nome]=WENO_dict[nome+' 2D'](API,
                                euler_equation_2D,
                                γ=γ)

    create_data(
        label,
        N,
        nome,
        U0,
        WENOs[nome],
        0.001,
        t_final,
        cfl,
        Δx,
        Δy,
        GhostPointsX,
        GhostPointsY,
        NullForce,
        continue_flag=False)
    create_movie(
        label,
        N,
        nome,
        x,
        y,
        save_dir=label+'Zoomed',
        figsize=(12,9),
        vmin=2,
        vmax=20,
        levels=100,
        xlim=(2.0,3.0),
        ylim=(0,0.6),
        colorbar=False,
        start_time=130)
    create_movie(
        label,
        N,
        nome,
        x,
        y,
        figsize=(3*8,8),
        vmin=1,
        vmax=22,
        levels=100,
        xlim=(0,3),
        ylim=None)

WENO-Z
Tempo inicial: 0.0


In [9]:
print(WENOs[nome].Sim_step.pretty_printed_concrete_signatures())

Sim_step_graph(u, Δt, Δx, Δy, fronteiraX=<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001DD85FE3C40>, fronteiraY=<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001DD85FE3E50>, Force=<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001DD85FE3C10>, t)
  Args:
    u: float64 Tensor, shape=(4, 241, 959)
    Δt: float64 Tensor, shape=(1, 1, 1)
    Δx: float64 Tensor, shape=()
    Δy: float64 Tensor, shape=()
    t: float64 Tensor, shape=(1, 1, 1)
  Returns:
    float64 Tensor, shape=(4, 241, 959)


RODAR SÓ ATÉ AQUI!

In [ ]:
γ = 7.0/5.0
θ = API_Numpy.pi/6.0
N = 400
x0=1/6
x, y, Δx, Δy, U0 = CondiçãoInicialShockEntropy2D_N(N, γ,θ)
cfl = 0.5
t_final = 1.8

GhostPointsX=ShockEntropy2DGhostPointsX(γ,Δx,y,θ)
GhostPointsY=ShockEntropy2DGhostPointsY()

WENOs={}

for name in WENOs_list:
    WENOs[name]=WENO_dict[name+' 2D'](API,
                            euler_equation_2D,
                            γ=γ)

In [ ]:
for name in WENOs_list:
    create_data(
        'ShockEntropy',
        N,
        name,
        U0,
        WENOs[name],
        0.01,
        t_final,
        cfl,
        Δx,
        Δy,
        GhostPointsX,
        GhostPointsY,
        NullForce,
        continue_flag=True)

    create_movie(
        'ShockEntropy',
        N,
        name,
        x,
        y,
        figsize=(2*8,8),
        vmin=0.75,
        vmax=4.75,
        levels=15,
        xlim=None,
        ylim=None)

WENO-Z (BI)
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Z                           
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Zp                          
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Z (MS)                      
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Zp (MS)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Zp (BI)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-JS (BI)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-ZC (MS)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-ZC (BI)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
